# Final Project – Integer / Linear Programming

Course: Optimization (Linear and Integer Programming)

Students  
- Francisco Alejandro Pérez Heredia – 2226996  
- Tadeo Alejandro Velazquez Vega – 2226980


In [1]:
import sympy as sp
import numpy as np
import pandas as pd


## Introduction

This final project studies a facility location problem for Community Centers for Digital Innovation (CCDI) in the Metropolitan Area of Monterrey (MAM), in the state of Nuevo León, Mexico.

The project is framed as a decision problem faced by the Dirección de Innovación y Tecnología Educativa of the Ministry of Education of Nuevo León. As part of a digital inclusion program, this public agency wants to deploy new CCDI where citizens can access computers, internet and basic digital skills training.

The aim of the first deployment phase is not to open a CCDI in every municipality, but to install a limited number of regional hubs that will serve several municipalities around them. In this phase the agency decided, for organisational and staffing reasons, to open at most three CCDI that will be operated directly by the state. The number three comes from the available teams and budget for three complete infrastructure packages; smaller computer rooms in schools and libraries remain managed by other programs and are not included in this model.

From the point of view of optimization, the problem is formulated as a mixed-integer linear programming (MILP) location–allocation model:

- Location decision: in which municipalities to open the large CCDI hubs.  
- Allocation decision: which municipality population is served by each open CCDI.

The objective is to minimise a proxy of the total travel effort of the users, under capacity and equity-by-region constraints. The model is solved with integer programming methods (LP relaxation and branch-and-bound), and the results are analysed critically to see if they make sense as a public policy proposal.


## Description of the case study

The Metropolitan Area of Monterrey (MAM) includes several municipalities with strong social and economic differences. In this project we consider six of them:

- Monterrey  
- Guadalupe  
- Apodaca  
- San Nicolás de los Garza  
- Santa Catarina  
- General Escobedo  

At the moment of the project we assume that there are no CCDI of this specific program yet. There are existing computer rooms in schools and libraries, but they are managed by other initiatives and are considered out of the scope. The Ministry wants to decide where to locate three new large CCDI that will operate as regional hubs for digital inclusion.

For each municipality we use the total population from the 2020 Mexican census and we estimate that 15 percent of the inhabitants are potential active users of the CCDI (students, teachers, young adults and job seekers). This factor is the same for all municipalities to keep the model simple and transparent.

Two extra policy requirements come from the priority maps of the state development plan:

- the east / north-east corridor (Guadalupe and Apodaca), and  
- the north / west belt (San Nicolás de los Garza, General Escobedo and Santa Catarina)  

are identified as zones with high population and relatively lower access to higher education institutions. To reflect this in the model, the agency asks that at least one CCDI must be located in each of those two macro-zones.

Physically, whenever a municipality is selected as a CCDI location, the center is assumed to be built next to the city council building of that municipality (that is, near the presidencia municipal). This convention is used both for the distance data and for the interpretation of the solution.


## Data Description

We work with six municipalities of the MAM. For each of them we use:

- Total population 2020 (INEGI census data, approximated).  
- Potential users: 15 percent of population.  
- Region label: Center, East, North-east, North, West (only for interpretation).  
- Pairwise distances between municipalities in kilometres.

Distances are approximate driving distances measured with Google Maps between the city council building of municipality i and the city council building of municipality j, rounded to the nearest kilometre. The idea is that each CCDI will be located next to the city council of its municipality, so these distances represent how far people must travel from one municipality to reach the CCDI placed by another municipality.


In [ ]:
import pandas as pd

population = {
    "Monterrey": 1142994,
    "Guadalupe": 643143,
    "Apodaca": 656464,
    "San Nicolás de los Garza": 412199,
    "Santa Catarina": 306322,
    "General Escobedo": 454957
}

demand = {m: int(round(p * 0.15)) for m, p in population.items()}

region = {
    "Monterrey": "Center",
    "Guadalupe": "East",
    "Apodaca": "North-east",
    "San Nicolás de los Garza": "North",
    "Santa Catarina": "West",
    "General Escobedo": "North"
}

df = pd.DataFrame({
    "municipality": list(population.keys()),
    "population_2020": list(population.values()),
    "demand_users": [demand[m] for m in population.keys()],
    "region": [region[m] for m in population.keys()]
})

df


## Problem Definition

We model the case as a location–allocation MILP. We use short names for the municipalities:

- Mon = Monterrey, Gua = Guadalupe, Apo = Apodaca  
- Sn = San Nicolás de los Garza, Sc = Santa Catarina, Esc = General Escobedo  

Each possible CCDI, if opened in a municipality, is assumed to be built next to the city council of that municipality. Since the distance data are measured between city councils, the objective function directly reflects the travel effort from the population of one municipality to the CCDI placed at another city council.

### Decision variables

Location variables (one for each municipality):

\[
y_{\text{Mon}}, y_{\text{Gua}}, y_{\text{Apo}}, y_{\text{Sn}}, y_{\text{Sc}}, y_{\text{Esc}} \in \{0,1\}
\]

For example, y_{Mon} = 1 means that a CCDI is opened next to the city council of Monterrey in this first deployment phase.

Assignment variables:

\[
x_{ij} =
\begin{cases}
1 & \text{if demand of municipality } i \text{ is served by a CCDI next to the city council of } j \\
0 & \text{otherwise}
\end{cases}
\]

### Parameters

- D_i: demand of potential users in municipality i (15 percent of its population).  
- c_{ij}: driving distance in km between the city council of municipality i and the city council of municipality j.  
- C = 200000: maximum capacity of one CCDI (users).  
- B = 3: maximum number of CCDI hubs that the Ministry decided to open in phase I.  

### Objective function

We minimise the total travel cost (distance times potential users):

$$
\min Z = \sum_i \sum_j c_{ij} D_i x_{ij}
$$

### Constraints

1. Unique assignment

$$
\sum_j x_{ij} = 1 \quad \forall i
$$

2. Assignment only to open centers

$$
x_{ij} \le y_j \quad \forall i,j
$$

3. Capacity of each CCDI

$$
\sum_i D_i x_{ij} \le C\,y_j \quad \forall j
$$

4. Maximum number of centers (planning decision for phase I)

$$
y_{\text{Mon}} + y_{\text{Gua}} + y_{\text{Apo}} + y_{\text{Sn}} + y_{\text{Sc}} + y_{\text{Esc}} \le 3
$$

5. Equity east / north-east priority zone

$$
y_{\text{Gua}} + y_{\text{Apo}} \ge 1
$$

6. Equity north / west priority zone

$$
y_{\text{Sn}} + y_{\text{Esc}} + y_{\text{Sc}} \ge 1
$$

7. Integrality

$$
x_{ij} \in \{0,1\}, \quad
y_{\text{Mon}}, y_{\text{Gua}}, y_{\text{Apo}}, y_{\text{Sn}}, y_{\text{Sc}}, y_{\text{Esc}} \in \{0,1\}
$$

This structure corresponds to a fixed-p facility location or p-median type problem: the number of centers is fixed by policy for phase I, and the optimisation decides where to place them and how to allocate demand.


In [ ]:
municipalities = list(population.keys())

distances = {
    ("Monterrey", "Monterrey"): 0,
    ("Monterrey", "Guadalupe"): 7,
    ("Monterrey", "Apodaca"): 19,
    ("Monterrey", "San Nicolás de los Garza"): 9,
    ("Monterrey", "Santa Catarina"): 13,
    ("Monterrey", "General Escobedo"): 18,

    ("Guadalupe", "Guadalupe"): 0,
    ("Guadalupe", "Apodaca"): 15,
    ("Guadalupe", "San Nicolás de los Garza"): 11,
    ("Guadalupe", "Santa Catarina"): 22,
    ("Guadalupe", "General Escobedo"): 24,

    ("Apodaca", "Apodaca"): 0,
    ("Apodaca", "San Nicolás de los Garza"): 17,
    ("Apodaca", "Santa Catarina"): 28,
    ("Apodaca", "General Escobedo"): 21,

    ("San Nicolás de los Garza", "San Nicolás de los Garza"): 0,
    ("San Nicolás de los Garza", "Santa Catarina"): 18,
    ("San Nicolás de los Garza", "General Escobedo"): 12,

    ("Santa Catarina", "Santa Catarina"): 0,
    ("Santa Catarina", "General Escobedo"): 25,

    ("General Escobedo", "General Escobedo"): 0
}

for i in municipalities:
    for j in municipalities:
        if (i, j) not in distances and (j, i) in distances:
            distances[(i, j)] = distances[(j, i)]

list(distances.items())[:10]


## Problem Solution

To solve the MILP model we use the PuLP library for Python, which allows us to define linear and integer programming problems and send them to an external solver. The solver uses the simplex method for the linear relaxations and a branch-and-bound strategy to enforce integrality of the binary variables.

The implementation follows these steps:

1. Create the minimisation problem.  
2. Define the binary decision variables y_j (location) and x_ij (assignment).  
3. Add the objective function with distances and demands.  
4. Add all constraints: unique assignment, assignment only if open, capacity, maximum number of centres and the two equity constraints.  
5. Call the solver and examine the optimal solution.


In [ ]:
!pip install pulp -q

import pulp as lp

D = demand
C = 200000
B = 3

prob = lp.LpProblem("Location_CCDI_Monterrey", lp.LpMinimize)

y = lp.LpVariable.dicts("y", municipalities, lowBound=0, upBound=1, cat=lp.LpBinary)

x = lp.LpVariable.dicts(
    "x",
    (municipalities, municipalities),
    lowBound=0,
    upBound=1,
    cat=lp.LpBinary
)

prob += lp.lpSum(
    distances[(i, j)] * D[i] * x[i][j]
    for i in municipalities
    for j in municipalities
), "Total_travel_cost"

for i in municipalities:
    prob += lp.lpSum(x[i][j] for j in municipalities) == 1, f"Unique_assignment_{i}"

for i in municipalities:
    for j in municipalities:
        prob += x[i][j] <= y[j], f"Assign_only_if_open_{i}_{j}"

for j in municipalities:
    prob += lp.lpSum(D[i] * x[i][j] for i in municipalities) <= C * y[j], f"Capacity_{j}"

prob += lp.lpSum(y[j] for j in municipalities) <= B, "Max_number_of_centers"

prob += y["Guadalupe"] + y["Apodaca"] >= 1, "Equity_east_northeast"

prob += (
    y["San Nicolás de los Garza"]
    + y["General Escobedo"]
    + y["Santa Catarina"]
) >= 1, "Equity_north_west"

prob.solve()

print("Status of the solution:", lp.LpStatus[prob.status])
print("\nOpened centers (y_j = 1):")
for j in municipalities:
    print(f"{j:25s} -> {int(y[j].varValue)}")

print("\nAssignments x_ij = 1 (municipality i is served by CCDI in j):")
for i in municipalities:
    for j in municipalities:
        if x[i][j].varValue > 0.5:
            print(f"{i:25s} is served by {j}")

print("\nOptimal value of the objective function (total travel cost):",
      lp.value(prob.objective))


## Solution Validation and Analysis

In this section we present the optimal solution obtained by the solver and we analyse it in detail.

### Explicit solution

For one run of the model the optimal location decision is:

- one CCDI in Monterrey (y_Mon = 1)  
- one CCDI in Apodaca (y_Apo = 1)  
- one CCDI in San Nicolás de los Garza (y_Sn = 1)  
- no CCDI in Guadalupe, Santa Catarina or General Escobedo (their y variables are zero)

So exactly three centres are opened, which matches the phase I planning limit B = 3. In geographical terms, the CCDI are located next to the city council buildings of Monterrey, Apodaca and San Nicolás de los Garza.

The assignment decisions x_ij are:

- Monterrey is served by the CCDI next to the city council of Monterrey.  
- Guadalupe is served by the CCDI next to the city council of Apodaca.  
- Apodaca is served by the CCDI next to the city council of Apodaca.  
- San Nicolás de los Garza is served by the CCDI next to its own city council.  
- Santa Catarina is served by the CCDI next to the city council of San Nicolás de los Garza.  
- General Escobedo is served by the CCDI next to the city council of San Nicolás de los Garza.  

The optimal value of the objective function (total travel cost = distance × demand) is around

$$
Z^{*} \approx 2.6 \times 10^{6} \text{ km·users}
$$

which is not directly interpretable for citizens but is useful to compare alternative scenarios.

### Validation of constraints

The main constraints are checked as follows:

- Unique assignment: each municipality appears exactly once in the assignment list, so the equation sum_j x_ij = 1 holds for all i.  
- Capacity: the total demand assigned to each open CCDI is below the capacity C = 200000 users, so no centre is overloaded.  
- Maximum number of centres: there are exactly three locations with y_j = 1, so the planning constraint is satisfied with equality.  
- Equity east / north-east: Apodaca is open, so y_Gua + y_Apo >= 1 holds.  
- Equity north / west: San Nicolás de los Garza is open, so y_Sn + y_Esc + y_Sc >= 1 is also satisfied.

The results are validated carefully, and the analysis is critical and not only mechanical.

### Critical discussion

Even if three CCDI are enough to cover all the potential demand from the capacity point of view, the limit of three hubs is not interpreted as a pure budget constraint inside the model. Instead, it represents a policy decision for phase I: the Ministry decided to open three large regional CCDI, and the optimisation helps to choose their locations.

The solution shows an interesting pattern:

1. Monterrey and Apodaca appear as natural hubs for the central and east / north-east parts of the MAM, because they concentrate high demand and are relatively central.  
2. San Nicolás de los Garza acts as a hub for the north / west part, serving itself, Santa Catarina and General Escobedo.  
3. Some municipalities do not receive a CCDI in their own territory (Guadalupe, Santa Catarina, General Escobedo), but are served by nearby hubs. This can be politically sensitive even if it is optimal in terms of distance.  
4. If the capacity C were reduced or the number of hubs B increased, the model would probably open more CCDI and reduce travel distances for the outer municipalities. This kind of sensitivity analysis is important before taking a final political decision.

Overall, the solution is coherent with the structure of the model and reveals the tension between efficiency (minimising travel cost) and territorial equity.


In [ ]:
assign_counts = {i: 0 for i in municipalities}
for i in municipalities:
    for j in municipalities:
        if x[i][j].varValue > 0.5:
            assign_counts[i] += 1

print("Assignments per municipality (should be 1 each):")
print(assign_counts)

capacity_use = {j: 0 for j in municipalities}
for j in municipalities:
    for i in municipalities:
        if x[i][j].varValue > 0.5:
            capacity_use[j] += D[i]

print("\nCapacity usage per centre (must be <= 200000 if y_j = 1):")
print(capacity_use)

num_open_centers = sum(int(y[j].varValue) for j in municipalities)
print("\nNumber of opened centres:", num_open_centers)

east_northeast = int(y["Guadalupe"].varValue) + int(y["Apodaca"].varValue)
north_west = (
    int(y["San Nicolás de los Garza"].varValue)
    + int(y["General Escobedo"].varValue)
    + int(y["Santa Catarina"].varValue)
)

print("\nEquity east / north-east (should be >= 1):", east_northeast)
print("Equity north / west (should be >= 1):", north_west)


## Conclusion

In this final project we modeled and solved a location–allocation problem for Community Centers for Digital Innovation in the Monterrey Metropolitan Area. The model combines elements from linear programming and integer programming, using binary variables and several constraints on capacity, equity and assignment of demand.

By using real population data and documented assumptions for demand and distances between city councils, the optimisation model gives a concrete proposal of where to open the CCDI hubs and how to assign each municipality to them. The use of a mixed-integer linear programming solver with branch-and-bound is necessary, because the structure of the problem is not simple and the best solution can not be found just by inspection.

The methodology can be extended to more municipalities, different types of facilities, or additional criteria such as installation costs, marginalisation indexes or priorities for vulnerable groups. In that sense, the project illustrates how the techniques of linear and integer programming studied in the course can support real decision making in public policies and educational innovation, not only in theory but also in practical applications.
